In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# clone YOLOv5 repository
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
!git reset --hard fbe67e465375231474a2ad80a4389efc77ecff99

Cloning into 'yolov5'...
remote: Enumerating objects: 14887, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 14887 (delta 0), reused 1 (delta 0), pack-reused 14882
Receiving objects: 100% (14887/14887), 13.92 MiB | 30.78 MiB/s, done.
Resolving deltas: 100% (10239/10239), done.
/content/yolov5
HEAD is now at fbe67e4 Fix `OMP_NUM_THREADS=1` for macOS (#8624)


In [ ]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
from utils.downloads import attempt_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

     |████████████████████████████████| 1.6 MB 5.0 MB/s 
Setup complete. Using torch 1.13.0+cu116 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15109MB, multi_processor_count=40)


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="TBSZIcJvZGLN6tqEEfeO")
project = rf.workspace("tfnet-night-vision").project("mul")
version = project.version(4)
dataset = version.download("yolov5")

/content/yolov5
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 42 kB 770 kB/s 
     |████████████████████████████████| 145 kB 8.9 MB/s 
     |████████████████████████████████| 178 kB 71.0 MB/s 
     |████████████████████████████████| 54 kB 3.6 MB/s 
     |████████████████████████████████| 138 kB 57.6 MB/s 
     |████████████████████████████████| 67 kB 5.9 MB/s 
     |████████████████████████████████| 62 kB 1.8 MB/s 
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=e01bdabc465e4bbe164db6bf1dc9ec636802fc2b1b20369026d7ea76b85c475c
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e81f33f462d86a1d895fae0
Successfully built wget
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: certifi
    Found existing inst

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Mul-1 in yolov5pytorch:: 100%|██████████| 9593/9593 [00:03<00:00, 2823.10it/s]


names:
- Bird
- Drone
- Helicopter
- Plane
nc: 4
roboflow:
  license: CC BY 4.0
  project: mul
  url: https://universe.roboflow.com/night-vision-mmcmp/mul/dataset/1
  version: 1
  workspace: night-vision-mmcmp
test: ../test/images
train: Mul-1/train/images
val: Mul-1/valid/images


In [ ]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [ ]:
#this is the model configuration we will use for our tutorial
%cat //content/yolov5/models/yolov5-p2.yaml

cat: //content/yolov5/models/yolov5-p2.yaml: No such file or directory


In [ ]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [ ]:
%%writetemplate /content/yolov5/models/YOLOG-p2345.yaml

# parameters
nc: {num_classes}  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.5  # layer channel multiple
anchors: 3  # AutoAnchor evolves 3 anchors per P output layer

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, GhostConv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3Ghost, [128]],
   [-1, 1, GhostConv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3Ghost, [256]],
   [-1, 1, GhostConv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3Ghost, [512]],
   [-1, 1, GhostConv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3Ghost, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head with (P2, P3, P4, P5) outputs
head:
  [[-1, 1, GhostConv, [512, 5, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3Ghost, [512, False]],  # 13

   [-1, 1, GhostConv, [256, 5, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, C3Ghost, [256, False]],  # 17 (P3/8-small)

   [-1, 1, GhostConv, [128, 5, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 2], 1, Concat, [1]],  # cat backbone P2
   [-1, 1, C3Ghost, [128, False]],  # 21 (P2/4-xsmall)

   [-1, 1, GhostConv, [128, 5, 2]],
   [[-1, 18], 1, Concat, [1]],  # cat head P3
   [-1, 3, C3Ghost, [256, False]],  # 24 (P3/8-small)

   [-1, 1, GhostConv, [256, 5, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, C3Ghost, [512, False]],  # 27 (P4/16-medium)

   [-1, 1, GhostConv, [512, 5, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, C3Ghost, [1024, False]],  # 30 (P5/32-large)

   [[21, 24, 27, 30], 1, Detect, [nc, anchors]],  # Detect(P2, P3, P4, P5)
  ]

In [ ]:
# time its performance
%%time
%cd /content/yolov5/
#!python train.py --img 416 --batch 32 --epochs 200 --data {dataset.location}/data.yaml --cfg ./models/GAANet-p2345.yaml --weights '' --name GAANet-p2345_results  --cache
!python train.py --img 320 --batch-size -1 --epochs 600  --optimizer AdamW --data {dataset.location}/data.yaml --cfg ./models/YOLOG-p2345.yaml --hyp data/hyps/hyp.scratch-high.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOG-p2345_results  --cache

/content/yolov5
train: weights=, cfg=./models/GAANet-p2345.yaml, data=/content/yolov5/Mul-1/data.yaml, hyp=data/hyps/hyp.scratch-high.yaml, epochs=500, batch_size=-1, imgsz=320, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=AdamW, sync_bn=False, workers=8, project=/content/gdrive/MyDrive/Runs, name=GAANet-p2345_results, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, seed=0, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: ⚠️ YOLOv5 is out of date by 413 commits. Use `git pull` or `git clone https://github.com/ultralytics/yolov5` to update.
YOLOv5 🚀 v6.1-306-gfbe67e4 Python-3.8.16 torch-1.13.0+cu116 CUDA:0 (Tesla T4, 15110MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentu

In [ ]:
#Later, if it is stopped prematurely and I need to resume, I use
#%%time
#%cd /content/yolov5/
#!python train.py --resume "/content/gdrive/MyDrive/Runs/GAANet-p34_results/weights/last.pt"
#!python train.py --img 320 --batch-size -1 --data {dataset.location}/data.yaml --hyp data/hyps/hyp.scratch-high.yaml --cfg ./models/YOLOG-p2345.yaml --weights '' --project "/content/gdrive/MyDrive/Runs"  --name YOLOG-p2345_results --cache  --resume "/content/gdrive/MyDrive/Runs/YOLOG-p2345_results/weights/last.pt"

In [ ]:
# we can also output some older school graphs if the tensor board isn't working for whatever reason...
from utils.plots import plot_results  # plot results.txt as results.png
Image(filename='/content/yolov5/runs/train/YOLOG-p2345_results/results.png', width=1000)  # view results.png

In [ ]:
%cd /content/yolov5/
!python detect.py --weights runs/train/YOLOG-p2345_results/weights/best.pt --img 320 --conf 0.4 --source /content/yolov5/Mul-1/valid/images
#--visualize

In [ ]:
!python val.py --task study --weights runs/train/YOLOG-p2345_results/weights/best.pt--data {dataset.location}/data.yaml --iou 0.5 --save_txt

In [ ]:
!python val.py --task speed --weights runs/train/YOLOG-p2345_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --save-txt

In [ ]:
!python val.py --weights runs/train/YOLOG-p2345_results/weights/best.pt --data {dataset.location}/data.yaml --iou 0.5 --img 320 --verbose

In [ ]:
##I have created a zip file:
!zip -r /content/GAANet-p2345.zip /content/yolov5/runs/GAANet-p2345_results2
##Than I have downloded that zip file:
from google.colab import files
files.download("/content/GAANet-p2345.zip")

In [ ]:
#### Visulize each layers feature map
#######Run this cell to know the whats inside every lare feature
#### stop it ur self after one image is done
## Print whole folder feature maps and save them
#!python detect.py --weights /content/yolov5/mfnets.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images --visualize
####### Only single image
#!python detect.py --weights runs/train/GAANet-p2345_results/weights/best.pt --img 416 --conf 0.4 --source /content/yolov5/Uav-7/test/images/0094_jpg.rf.09856e96b7f9f77c2bc502452ec20d77.jpg --visualize